# TP 5 : Tableaux et Complexité

## Petites questions

1. Écrire une fonction `somme` pour calculer la somme des éléments d'un tableau d'entiers.  
2. Écrire une fonction `maximum` pour calculer le maximum d'un tableau d'entiers. On pourra utiliser la fonction `max` renvoyant le maximum de 2 nombres.  
3. Écrire une fonction `list_of_array` transformant un tableau en liste.  
4. Tester si un tableau est trié par ordre croissant.

In [6]:
(*question 1*)
let somme t =
    let s = ref 0 in
    for i=0 to ((Array.length t)-1) do
        s:= !s + t.(i)
    done;
    !s;;

somme [|1;7;3|];;

val somme : int array -> int = <fun>


- : int = 11


In [5]:
(*question 2*)
let maximum t =
    let m = ref 0 in
    for i=0 to ((Array.length t)-1) do
        if t.(i)> !m then m := t.(i)      
    done;
    !m;;
    
maximum [|-1;8;4|];;

val maximum : int array -> int = <fun>


- : int = 8


In [3]:
(*question 3*)
let rec liste t n =
    if n = -1 then []
    else t.(n)::(liste t (n-1));;

let list_of_array t =
    let n = (Array.length t)-1 in
    List.rev(liste t n);;

list_of_array [|1;2;3;4|];;

val liste : 'a array -> int -> 'a list = <fun>


val list_of_array : 'a array -> 'a list = <fun>


- : int list = [1; 2; 3; 4]


In [4]:
(*question 4*)
let ordre_croissant t =
    for i = 0 to ((Array.length t)-1) do
        if t.(i)>t.(i+1) then false
    done;
    true;;
    

error: compile_error

## Maximum local dans un tableau

Un maximum local dans un tableau `t` est un indice `i` tel que `t.(i) >= t.(i+1)` et `t.(i) >= t.(i-1)`. Pour les extrémités, qu'une seule de ces conditions doit être vérifiée (si `t.(i-1)` ou `t.(i+1)` n'existe pas).  
1. Montrer qu'il existe forcément un minimum local dans `t`.  
2. Écrire une fonction `max_local1` trouvant un maximum local dans un tableau en regardant chaque élément un par un (recherche séquentielle). Quelle est sa complexité ?  
3. Ecrire une fonction `max_local2` faisant la même chose mais avec une méthode par dichotomie (en divisant par 2 la taille du problème à chaque itération), pour avoir une complexité logarithmique.  
*Aide* : soit `t.(m)` le milieu du tableau. Que peut-on dire si `t.(m) < t.(m+1)` ? Si `t.(m) < t.(m-1)` ?

In [11]:
(*question 2*)
let max_local1 t = let m = ref (-1) in
    if t.(0) >= t.(1) then m:=0;
    let n = Array.length t in
    if t.(n-1) >= t.(n-2) then m:= (n-1);
    for i=1 to n-2 do
        if t.(i)>= max t.(i-1) t.(i+1)
        then m := i
    done;
    !m;;

max_local1 [|1;2;5;4|];;
    

val max_local1 : 'a array -> int = <fun>


- : int = 2


In [32]:
let max_local2 t =
    let rec aux i j = 
        let m = (i+j)/2 in 
        if ( m=0 || t.(m) >= t.(m-1)) && (m=(Array.length t)-1 || t.(m) >= t.(m+1)) then m
        else if t.(m) <= t.(m+1) then aux (m+1) j
        else aux i (m-1)
in aux 0 (Array.length t);;

max_local2 [|2;3;103;2;3;4;5;6;7|]

val max_local2 : 'a array -> int = <fun>


- : int = 8


##### Tri par comptage

Écrire une fonction `tri_comptage : ’a array -> unit` qui trie un tableau `t` de taille $n$ dont les entrées sont des entiers entre 0
et $M$ (où $M$ est le maximum de `t`), en complexité O($n + M$).  
Pour cela :  
- Créer un autre tableau `compte` de taille $M + 1$ (avec `Array.make`), initialement rempli de 0
- Remplir `compte` pour que `compte.(i)` contienne le nombre d'apparitions de `i` dans `t`
- Recopier les éléments de `compte` dans `t` dans l'ordre croissant pour obtenir un tableau trié

In [9]:
let tri_comptage t = 
    let m = maximum t in
    let compte = Array.make (m+1) 0 in
    for i= 0 to Array.length t-1 do
        compte.(t.(i)) <- compte.(t.(i)) +1
    done;
    let indice = ref 0 in
    for j = 0 to m do
        for k = 1 to compte.(j) do
            t.(!indice) <- j;
            incr indice
        done;
    done;
    t;;
    

tri_comptage [|4;6;9;3;1;2;1;1;8;6;9;|]

val tri_comptage : int array -> int array = <fun>


- : int array = [|1; 1; 1; 2; 3; 4; 6; 6; 8; 9; 9|]


interrupt: intterupt

## Tranche maximum

Ecrire une fonction `tranche_max : int array -> int` qui renvoie en O($n$) la somme maximum d'éléments consécutifs d'un tableau de taille $n$. Par exemple, `tranche_max [|1; -2; 6; -3; 2; 4; -8; 7|]` doit renvoyer $9$, correspondant aux éléments 6; -3; 2; 4.  

*Indice* : parcourir le tableau `t` avec une boucle for. Stocker 2 variables `m` et `m_cur` telles que, au $i$ème passage dans la boucle for :  
- `m` contient la somme maximum d'éléments consécutifs parmi les $i$ premiers éléments du tableau
- `m_cur` contient la somme maximum d'éléments consécutifs finissant en `t.(i)`.  
Par exemple, lorsque `tranche_max [|1; -2; 6; -3; 2; 4; -8; 7|]` exécute la $3$ème itération de la boucle `for` (c'est à dire que -3 est considéré), `m` contient 6 (correspondant au seul élément 6) et `m_cur` contient 3 (correspondant à 6; -3).

In [7]:
let tranche_max t =
    let m = ref 0 in 
    let m_cur = ref 1 in
    let aux i =
    for a= i to array.length t-2 do 
    if t.(a)<t.(a+1) then m_cur := !m_cur + 1 && (if !m_cur > !m then m := !m +1)
    else m_cur := 1 && aux (i+1)
    done;
    !m;;
    
let tranche_max [|2;3;4;2;3;4;5;6;2;1|]

error: compile_error

## Inversions dans un tableau

Étant donné un tableau `t`, une inversion de `t` est un couple d'indices $(i, j)$ tels que $i < j$ et `t`.$(i)$ > `t`.$(j)$. Par exemple, `[|4; 1; 5; 2|]` possède 3 inversions: $(0, 1)$, $(0, 3)$ et $(2, 3)$.
- Écrire une fonction `inv` en O($n^2$) déterminant le nombre d'inversions d'un tableau de taille $n$.
- Écrire une fonction `inv_triee` telle que, si `l1` et `l2` sont deux listes triées par ordre croissant, renvoie le nombre de couples $(i, j)$ tels que `l1.(i) > l2.(j)`.
- (Difficile) Écrire une fonction `inv2` telle que, si `t` est un tableau de taille $n$, `inv2 t` renvoie le nombre d'inversions de `t` en complexité O($n\log(n)$).  
On modifiera le tri fusion pour renvoyer le nombre d'inversions en plus de la liste triée.